In [23]:
import csv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_score
from xgboost.sklearn import XGBRegressor
import xgboost as xgb

In [3]:
train = pd.read_csv("train.csv")
train_l = pd.read_csv("train_label.csv")
test = pd.read_csv("test.csv")

In [4]:
print(train.shape)
print(train_l.shape)
print(test.shape)

(91531, 33)
(640, 2)
(27859, 29)


In [33]:
df = train.copy()
del df["reservation_status_date"]
del df["reservation_status"]
del df["ID"]

In [6]:
print(train.columns)

Index(['ID', 'hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')


In [7]:
print(test.columns)

Index(['ID', 'hotel', 'lead_time', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'meal', 'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type',
       'required_car_parking_spaces', 'total_of_special_requests'],
      dtype='object')


In [8]:
print(set(train.columns) - set(test.columns))

{'is_canceled', 'reservation_status_date', 'adr', 'reservation_status'}


In [51]:
def change_date(df):
    date = []
    for i in range(df.shape[0]):
        day = str(df["arrival_date_day_of_month"][i])
        if df["arrival_date_day_of_month"][i] < 10:
            day = "0" + day
        if df["arrival_date_month"][i] == "January":
            s = str(df["arrival_date_year"][i]) + "-01-" + day
        elif df["arrival_date_month"][i] == "February":
            s = str(df["arrival_date_year"][i]) + "-02-" + day
        elif df["arrival_date_month"][i] == "March":
            s = str(df["arrival_date_year"][i]) + "-03-" + day
        elif df["arrival_date_month"][i] == "April":
            s = str(df["arrival_date_year"][i]) + "-04-" + day
        elif df["arrival_date_month"][i] == "May":
            s = str(df["arrival_date_year"][i]) + "-05-" + day
        elif df["arrival_date_month"][i] == "June":
            s = str(df["arrival_date_year"][i]) + "-06-" + day
        elif df["arrival_date_month"][i] == "July":
            s = str(df["arrival_date_year"][i]) + "-07-" + day
        elif df["arrival_date_month"][i] == "August":
            s = str(df["arrival_date_year"][i]) + "-08-" + day
        elif df["arrival_date_month"][i] == "September":
            s = str(df["arrival_date_year"][i]) + "-09-" + day
        elif df["arrival_date_month"][i] == "October":
            s = str(df["arrival_date_year"][i]) + "-10-" + day
        elif df["arrival_date_month"][i] == "November":
            s = str(df["arrival_date_year"][i]) + "-11-" + day
        elif df["arrival_date_month"][i] == "December":
            s = str(df["arrival_date_year"][i]) + "-12-" + day
        date.append(s)
        
    return date

In [52]:
arr_date = change_date(df)

In [34]:
df2 = df.copy()
del df2["arrival_date_year"]
del df2["arrival_date_month"]
del df2["arrival_date_day_of_month"]
train_dum = pd.get_dummies(df2)

In [35]:
display(train_dum)

,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,...,assigned_room_type_K,assigned_room_type_L,assigned_room_type_P,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,0,342,27,0,0,2,0.0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,257,27,0,2,1,0.0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,257,27,0,2,2,0.0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,257,27,0,2,2,0.0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,0,257,27,0,2,2,0.0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,1,19,13,0,2,2,2.0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
91527,0,28,13,0,2,2,0.0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
91528,0,2,13,0,1,2,0.0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
91529,0,30,13,3,7,2,0.0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


### daily revenue = adr * number of days * is cancel(opposite)

### Predict whether the reservation will be cancelled => logistic

In [13]:
can_y = train_dum["is_canceled"]
tmp = [train_dum["previous_cancellations"], train_dum["previous_bookings_not_canceled"], 
       train_dum["booking_changes"]]
can_x = pd.concat(tmp, axis = 1)

In [14]:
xcan_subtrain, xcan_val, ycan_subtrain, ycan_val = train_test_split(can_x, can_y, test_size=0.1, random_state=1024)

In [15]:
LR_a = 0.0
max_score = -10**10

accuracies1 = []
recalls1 = []
f1s1 = []
precisions1 = []

for i in np.logspace(-2,2,20):
    sk_current = LogisticRegression(C = 1/i).fit(xcan_subtrain, ycan_subtrain)
    sk_ypred = sk_current.predict(xcan_val)
    
    acc = accuracy_score(ycan_val, sk_ypred)
    accuracies1.append(acc)
    recalls1.append(recall_score(ycan_val, sk_ypred))
    f1s1.append(f1_score(ycan_val, sk_ypred))
    precisions1.append(precision_score(ycan_val, sk_ypred))
    
    if acc > max_score:
        max_score = acc
        LR_a = i

In [16]:
print(max_score)
print(LR_a)

0.7003495739567402
0.01


### Predict whether the reservation will be cancelled => XGBoost

In [177]:
best_s_can = 0
best_d_can = 0
max_score_can = 0

for d in max_d:
    for s in sub_s:
        # --------------------------------------------------------- #
        # train
        xgbc = XGBClassifier(booster="gbtree",
                     max_depth = d,
                     learning_rate=0.05,
                     n_estimators=200,
                     subsample = s, 
                     objective='binary:logistic', 
                     eval_metric = 'error')
        xgbc.fit(xcan_subtrain, ycan_subtrain)

        # valid
        ypred = xgbc.predict(xcan_val)
        acc = accuracy_score(ycan_val, ypred)
        
        if acc > max_score_can:
            print("! update best !")
            best_s_can = s
            best_d_can = d
            max_score_can = acc
        # --------------------------------------------------------- #
        print("s: %f, d: %d, acc: %f, \n"%(s,d,acc))
print("best s: %f, best d: %d, acc: %f"%(best_s_can,best_d_can,max_score_can))

! update best !
s: 0.800000, d: 5, acc: 0.699913, 

! update best !
s: 0.850000, d: 5, acc: 0.700022, 

s: 0.950000, d: 5, acc: 0.699913, 

s: 0.800000, d: 7, acc: 0.699913, 

s: 0.850000, d: 7, acc: 0.699913, 

s: 0.950000, d: 7, acc: 0.699913, 

s: 0.800000, d: 10, acc: 0.699913, 

s: 0.850000, d: 10, acc: 0.699803, 

s: 0.950000, d: 10, acc: 0.699803, 

best s: 0.850000, best d: 5, acc: 0.700022


In [185]:
best_l_can = 0
best_n_can = 0
max_score_can2 = 0

for l in l_r:
    for n in n_est:
        # --------------------------------------------------------- #
        # train
        xgbc = XGBClassifier(booster="gbtree",
                     max_depth = best_d_can,
                     learning_rate=l,
                     n_estimators=n,
                     subsample = best_s_can, 
                     objective='binary:logistic', 
                     eval_metric = 'error')
        xgbc.fit(xcan_subtrain, ycan_subtrain)

        # valid
        ypred = xgbc.predict(xcan_val)
        acc = accuracy_score(ycan_val, ypred)
        
        if acc > max_score_can2:
            print("! update best !")
            best_l_can = l
            best_n_can = n
            max_score_can2 = acc
        # --------------------------------------------------------- #
        print("l: %f, n: %d, score: %f, \n"%(l,n,acc))
print("best l: %f, best n: %d, score: %f"%(best_l_can,best_n_can,max_score_can2))

! update best !
l: 0.050000, n: 200, score: 0.700022, 

l: 0.050000, n: 500, score: 0.699803, 

l: 0.050000, n: 800, score: 0.699694, 

l: 0.050000, n: 1000, score: 0.699694, 

l: 0.075000, n: 200, score: 0.699803, 

l: 0.075000, n: 500, score: 0.699803, 

l: 0.075000, n: 800, score: 0.699694, 

l: 0.075000, n: 1000, score: 0.699694, 

l: 0.100000, n: 200, score: 0.699803, 

l: 0.100000, n: 500, score: 0.699694, 

l: 0.100000, n: 800, score: 0.699694, 

l: 0.100000, n: 1000, score: 0.699694, 

best l: 0.050000, best n: 200, score: 0.700022


### Use all data

In [204]:
can_y2 = train_dum["is_canceled"]
tt = df2.copy()
del tt["country"]
del tt["is_canceled"]
all_can = pd.get_dummies(tt)

In [209]:
xcan_sub2, xcan_val2, ycan_sub2, ycan_val2 = train_test_split(all_can, can_y2, test_size=0.1, random_state=1024)

In [212]:
display(ycan_sub2)

23646    0
38433    0
77186    0
32003    1
1348     0
        ..
4961     0
601      0
19045    0
74641    0
71611    1
Name: is_canceled, Length: 82377, dtype: int64

In [213]:
best_s_can3 = 0
best_d_can3 = 0
max_score_can3 = 0

for d in max_d:
    for s in sub_s:
        # --------------------------------------------------------- #
        # train
        xgbc = XGBClassifier(booster="gbtree",
                     max_depth = d,
                     learning_rate=0.05,
                     n_estimators=200,
                     subsample = s, 
                     objective='binary:logistic', 
                    use_label_encoder =False,
                     eval_metric = 'error')
        xgbc.fit(xcan_sub2, ycan_sub2)

        # valid
        ypred = xgbc.predict(xcan_val2)
        acc = accuracy_score(ycan_val2, ypred)
        
        if acc > max_score_can3:
            print("! update best !")
            best_s_can3 = s
            best_d_can3 = d
            max_score_can3 = acc
        # --------------------------------------------------------- #
        print("s: %f, d: %d, acc: %f, \n"%(s,d,acc))
print("best s: %f, best d: %d, acc: %f"%(best_s_can3,best_d_can3,max_score_can3))

! update best !
s: 0.800000, d: 5, acc: 0.843456, 

s: 0.850000, d: 5, acc: 0.841599, 

s: 0.950000, d: 5, acc: 0.841053, 

! update best !
s: 0.800000, d: 7, acc: 0.855364, 

s: 0.850000, d: 7, acc: 0.853944, 

s: 0.950000, d: 7, acc: 0.852633, 

! update best !
s: 0.800000, d: 10, acc: 0.864212, 

s: 0.850000, d: 10, acc: 0.863120, 

! update best !
s: 0.950000, d: 10, acc: 0.865742, 

best s: 0.950000, best d: 10, acc: 0.865742


In [214]:
best_l_can3 = 0
best_n_can3 = 0
max_score_can4 = 0

for l in l_r:
    for n in n_est:
        # --------------------------------------------------------- #
        # train
        xgbc = XGBClassifier(booster="gbtree",
                     max_depth = best_d_can3,
                     learning_rate=l,
                     n_estimators=n,
                     subsample = best_s_can3, 
                     objective='binary:logistic', 
                     eval_metric = 'error')
        xgbc.fit(xcan_sub2, ycan_sub2)

        # valid
        ypred = xgbc.predict(xcan_val2)
        acc = accuracy_score(ycan_val2, ypred)
        
        if acc > max_score_can4:
            print("! update best !")
            best_l_can3 = l
            best_n_can3 = n
            max_score_can4 = acc
        # --------------------------------------------------------- #
        print("l: %f, n: %d, score: %f, \n"%(l,n,acc))
print("best l: %f, best n: %d, score: %f"%(best_l_can3,best_n_can3,max_score_can4))

! update best !
l: 0.050000, n: 200, score: 0.865742, 

! update best !
l: 0.050000, n: 500, score: 0.872624, 

! update best !
l: 0.050000, n: 800, score: 0.876447, 

l: 0.050000, n: 1000, score: 0.876447, 

l: 0.075000, n: 200, score: 0.869456, 

l: 0.075000, n: 500, score: 0.875901, 

! update best !
l: 0.075000, n: 800, score: 0.876557, 

l: 0.075000, n: 1000, score: 0.876229, 

l: 0.100000, n: 200, score: 0.870002, 

l: 0.100000, n: 500, score: 0.874809, 

! update best !
l: 0.100000, n: 800, score: 0.877540, 

l: 0.100000, n: 1000, score: 0.875137, 

best l: 0.100000, best n: 800, score: 0.877540


In [17]:
for_adr = train_dum.copy()
del for_adr["adr"]
adr_x = for_adr
adr_y = train_dum["adr"]

In [18]:
xadr_sub, xadr_val, yadr_sub, yadr_val = train_test_split(adr_x, adr_y, test_size=0.1, random_state=1024)

In [19]:
l_r = [0.05, 0.075, 0.1]
n_est = [200, 500, 800, 1000]
max_d=[5,7,10]
sub_s = [0.8, 0.85, 0.95]

In [20]:
XGBC_DT = dict()

In [26]:
best_s = 0
best_d = 0
max_score = 0

for d in max_d:
    for s in sub_s:
        # --------------------------------------------------------- #
        # train
        xgbc = XGBRegressor(booster="gbtree",
                     max_depth = d,
                     learning_rate=0.05,
                     n_estimators=200,
                     subsample = s)
        xgbc.fit(xadr_sub, yadr_sub)
        XGBC_DT[str(d)+'@'+str(s)] = xgbc

        # valid
        ypred = xgbc.predict(xadr_val)
        score = xgbc.score(xadr_val, yadr_val)  
        
        if score > max_score:
            print("! update best !")
            best_s = s
            best_d = d
            max_score = score
        # --------------------------------------------------------- #
        print("s: %f, d: %d, score: %f, \n"%(s,d,score))
print("best s: %f, best d: %d, score: %f"%(best_s,best_d,max_score))

! update best !
s: 0.800000, d: 5, score: 0.815118, 

s: 0.850000, d: 5, score: 0.813143, 

s: 0.950000, d: 5, score: 0.814488, 

! update best !
s: 0.800000, d: 7, score: 0.851001, 

s: 0.850000, d: 7, score: 0.849456, 

! update best !
s: 0.950000, d: 7, score: 0.852224, 

! update best !
s: 0.800000, d: 10, score: 0.871480, 

! update best !
s: 0.850000, d: 10, score: 0.873940, 

s: 0.950000, d: 10, score: 0.869937, 

best s: 0.850000, best d: 10, score: 0.873940


In [27]:
import joblib
joblib.dump(XGBC_DT[str(best_d)+'@'+str(best_s)], 'imb_XGBC')

['imb_XGBC']

In [28]:
XGBC_DT2 = dict()

In [30]:
best_l = 0
best_n = 0
max_score2 = 0

for l in l_r:
    for n in n_est:
        # --------------------------------------------------------- #
        # train
        xgbc = XGBRegressor(booster="gbtree",
                     max_depth = best_d,
                     learning_rate = l,
                     n_estimators = n,
                     subsample = best_s)
        xgbc.fit(xadr_sub, yadr_sub)
        XGBC_DT2[str(l)+'@'+str(n)] = xgbc

        # valid
        ypred = xgbc.predict(xadr_val)
        score = xgbc.score(xadr_val, yadr_val)  
        
        if score > max_score2:
            print("! update best !")
            best_l = l
            best_n = n
            max_score2 = score
        # --------------------------------------------------------- #
        print("l: %f, n: %d, score: %f, \n"%(l,n,score))
print("best l: %f, best n: %d, score: %f"%(best_l,best_n,max_score2))

! update best !
l: 0.050000, n: 200, score: 0.873940, 

! update best !
l: 0.050000, n: 500, score: 0.880936, 

! update best !
l: 0.050000, n: 800, score: 0.883001, 

! update best !
l: 0.050000, n: 1000, score: 0.883526, 

l: 0.075000, n: 200, score: 0.873833, 

l: 0.075000, n: 500, score: 0.879477, 

l: 0.075000, n: 800, score: 0.880434, 

l: 0.075000, n: 1000, score: 0.880445, 

l: 0.100000, n: 200, score: 0.877594, 

l: 0.100000, n: 500, score: 0.880874, 

l: 0.100000, n: 800, score: 0.881179, 

l: 0.100000, n: 1000, score: 0.880666, 

best l: 0.050000, best n: 1000, score: 0.883526


In [54]:
rev_y = train_l["label"]
op = [1]*train.shape[0]
tmp_x = train["adr"]*(op - train["is_canceled"])*(train["stays_in_weekend_nights"] + train["stays_in_week_nights"])
arr_df = pd.DataFrame(arr_date)
cat_x = pd.concat([arr_df, tmp_x], axis = 1)
cat_x.columns = ["arrDate", "rev"]
print(cat_x)

          arrDate         rev
0      2015-07-01   -0.000000
1      2015-07-01  150.104454
2      2015-07-01  149.092802
3      2015-07-01  152.752575
4      2015-07-01   98.823294
...           ...         ...
91526  2017-03-31    0.000000
91527  2017-03-31  -13.644203
91528  2017-03-31   90.814554
91529  2017-03-31  381.355651
91530  2017-03-31   58.196470

[91531 rows x 2 columns]


In [43]:
print(train_l["arrival_date"])

0      2015-07-01
1      2015-07-02
2      2015-07-03
3      2015-07-04
4      2015-07-05
          ...    
635    2017-03-27
636    2017-03-28
637    2017-03-29
638    2017-03-30
639    2017-03-31
Name: arrival_date, Length: 640, dtype: object


In [55]:
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list

In [56]:
uni_date = unique(arr_date)
print(len(uni_date))

640


In [77]:
blank = [0]*len(uni_date)

In [78]:
#sum up
for i in range(cat_x.shape[0]):
    for j in range(len(uni_date)):
        if cat_x["arrDate"][i] == uni_date[j]:
            blank[j] += cat_x["rev"][i]
            break

In [80]:
uni_df = pd.DataFrame(uni_date)
blank = pd.DataFrame(blank)
cat_uni = pd.concat([uni_df, blank], axis = 1)
cat_uni.columns = ["arrDate", "allRev"]

In [81]:
print(cat_uni)

        arrDate        allRev
0    2015-07-01  20311.186621
1    2015-07-02  16530.645277
2    2015-07-03  12966.714164
3    2015-07-04  17480.654256
4    2015-07-05  19591.458478
..          ...           ...
635  2017-03-27  26217.381380
636  2017-03-28  16185.177703
637  2017-03-29  24002.255525
638  2017-03-30  33095.297394
639  2017-03-31  36062.103164

[640 rows x 2 columns]


### Predict class

In [87]:
xcl = cat_uni["allRev"]
ycl = train_l["label"]
xcl_sub, xcl_val, ycl_sub, ycl_val = train_test_split(xcl, ycl, test_size=0.1, random_state=1024)
xcl_sub = xcl_sub.to_numpy().reshape(-1,1)
xcl_val = xcl_val.to_numpy().reshape(-1,1)

In [83]:
XGBC_DT3 = dict()

In [101]:
LR_a2 = 0.0
max_score2 = -10**10

accuracies2 = []

for i in np.logspace(-2,2,20):
    sk_current = LogisticRegression(C = 1/i, multi_class='auto').fit(xcl_sub, ycl_sub)
    sk_ypred = sk_current.predict(xcl_val)
    
    acc = accuracy_score(ycl_val, sk_ypred)
    accuracies2.append(acc)
    
    if acc > max_score2:
        max_score2 = acc
        LR_a2 = i

In [102]:
print(LR_a2)
print(max_score2)

0.01
0.296875


In [92]:
import xgboost as xgb

In [93]:
dsub = xgb.DMatrix(xcl_sub, label=ycl_sub)
dval = xgb.DMatrix(xcl_val, label=ycl_val)

In [114]:
best_s_cl = 0
best_d_cl = 0
max_score_cl = 0

for d in max_d:
    for s in sub_s:
        # --------------------------------------------------------- #
        # train
        param = {'max_depth':d, 'eta':1, 'objective':'multi:softmax', 'num_class':10, 'learning_rate':0.05, 
                'subsample':s, 'booster':"gbtree", 'eval_metric':'mlogloss'}
        num_round = 200
        bst = xgb.train(param, dsub, num_round)

        # valid
        ypred = bst.predict(dval)
        acc = accuracy_score(ycl_val, ypred)
        #score = xgb.score(xcl_val, ycl_val)  
        
        if acc > max_score_cl:
            print("! update best !")
            best_s_cl = s
            best_d_cl = d
            max_score_cl = acc
        # --------------------------------------------------------- #
        print("s: %f, d: %d, acc: %f, \n"%(s,d,acc))
print("best s: %f, best d: %d, acc: %f"%(best_s_cl,best_d_cl,max_score_cl))

! update best !
s: 0.800000, d: 5, acc: 1.000000, 

s: 0.850000, d: 5, acc: 1.000000, 

s: 0.950000, d: 5, acc: 1.000000, 

s: 0.800000, d: 7, acc: 1.000000, 

s: 0.850000, d: 7, acc: 1.000000, 

s: 0.950000, d: 7, acc: 1.000000, 

s: 0.800000, d: 10, acc: 1.000000, 

s: 0.850000, d: 10, acc: 1.000000, 

s: 0.950000, d: 10, acc: 1.000000, 

best s: 0.800000, best d: 5, acc: 1.000000


In [ ]:
best_s_cl = 0
best_d_cl = 0
max_score_cl = 0

for d in max_d:
    for s in sub_s:
        # --------------------------------------------------------- #
        # train
        param = {'max_depth':d, 'eta':1, 'objective':'multi:softmax', 'num_class':10, 'learning_rate':0.05, 
                'subsample':s, 'booster':"gbtree", 'eval_metric':'mlogloss'}
        num_round = 200
        bst = xgb.train(param, dsub, num_round)

        # valid
        ypred = bst.predict(dval)
        acc = accuracy_score(ycl_val, ypred)
        #score = xgb.score(xcl_val, ycl_val)  
        
        if acc > max_score_cl:
            print("! update best !")
            best_s_cl = s
            best_d_cl = d
            max_score_cl = acc
        # --------------------------------------------------------- #
        print("s: %f, d: %d, acc: %f, \n"%(s,d,acc))
print("best s: %f, best d: %d, acc: %f"%(best_s_cl,best_d_cl,max_score_cl))

In [116]:
best_l_cl = 0
best_n_cl = 0
max_score_cl2 = 0

for l in l_r:
    for n in n_est:
        # --------------------------------------------------------- #
        # train
        param = {'max_depth':best_d_cl, 'eta':1, 'objective':'multi:softmax', 'num_class':10, 
                 'learning_rate':l, 'subsample':best_s_cl, 'booster':"gbtree", 'eval_metric':'mlogloss'}
        num_round = n
        bst = xgb.train(param, dsub, num_round)

        # valid
        ypred = bst.predict(dval)
        acc = accuracy_score(ycl_val, ypred)
        
        if acc > max_score_cl2:
            print("! update best !")
            best_l_cl = l
            best_n_cl = n
            max_score_cl2 = acc
        # --------------------------------------------------------- #
        print("l: %f, n: %d, acc: %f, \n"%(l,n,acc))
print("best l: %f, best n: %d, acc: %f"%(best_l_cl,best_n_cl,max_score_cl2))

! update best !
l: 0.050000, n: 200, acc: 1.000000, 

l: 0.050000, n: 500, acc: 1.000000, 

l: 0.050000, n: 800, acc: 1.000000, 

l: 0.050000, n: 1000, acc: 1.000000, 

l: 0.075000, n: 200, acc: 1.000000, 

l: 0.075000, n: 500, acc: 1.000000, 

l: 0.075000, n: 800, acc: 1.000000, 

l: 0.075000, n: 1000, acc: 1.000000, 

l: 0.100000, n: 200, acc: 1.000000, 

l: 0.100000, n: 500, acc: 1.000000, 

l: 0.100000, n: 800, acc: 1.000000, 

l: 0.100000, n: 1000, acc: 1.000000, 

best l: 0.050000, best n: 200, acc: 1.000000


### Test

In [118]:
df_test = test.copy()
del df_test["ID"]

In [119]:
arr_date_test = change_date(df_test)

In [121]:
df2_test = df_test.copy()
del df2_test["arrival_date_year"]
del df2_test["arrival_date_month"]
del df2_test["arrival_date_day_of_month"]
test_dum = pd.get_dummies(df2_test)

In [125]:
tmpp = [test_dum["previous_cancellations"], test_dum["previous_bookings_not_canceled"], 
       test_dum["booking_changes"]]
xcan_test = pd.concat(tmpp, axis = 1)

In [215]:
#all_data
ttt = df2_test.copy()
del ttt["country"]
all_can_test =  pd.get_dummies(ttt)

In [219]:
#print(all_can.columns)
#print(all_can_test.columns)
print(set(all_can.columns) - set(all_can_test.columns))
print(set(all_can_test.columns) - set(all_can.columns))

{'market_segment_Undefined', 'adr', 'assigned_room_type_L', 'reserved_room_type_L', 'distribution_channel_Undefined'}
set()


In [220]:
col_name = list(set(all_can.columns) - set(all_can_test.columns))
for i in col_name:
    del all_can[i]

In [222]:
xgbc = XGBClassifier(booster="gbtree",
                     max_depth = best_d_can3,
                     learning_rate=best_l_can3,
                     n_estimators=best_n_can3,
                     subsample = best_s_can3, 
                     objective='binary:logistic', 
                     eval_metric = 'error')
xgbc.fit(all_can, can_y2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='error',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.1, max_delta_step=0,
              max_depth=10, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=800, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.95, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [223]:
ypred_can = xgbc.predict(all_can_test)

In [224]:
print(sum(ypred_can))

8759


In [186]:
xgbc = XGBClassifier(booster="gbtree",
                     max_depth = best_d_can,
                     learning_rate=best_l_can,
                     n_estimators=best_n_can,
                     subsample = best_s_can, 
                     objective='binary:logistic', 
                     eval_metric = 'error')
xgbc.fit(can_x, can_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='error',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.05, max_delta_step=0,
              max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=200, n_jobs=8,
              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=0.85, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [187]:
ypred_can = xgbc.predict(xcan_test)

In [188]:
print(sum(ypred_can))

4


In [130]:
logr = LogisticRegression(C = 1/LR_a)
logr.fit(can_x, can_y)
ypred_can = logr.predict(xcan_test)

In [142]:
print(adr_x.columns)
print(test_dum.columns)
print(set(adr_x.columns) - set(test_dum.columns))
print(set(test_dum.columns) - set(adr_x.columns))

Index(['lead_time', 'arrival_date_week_number', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled',
       ...
       'assigned_room_type_I', 'assigned_room_type_K', 'assigned_room_type_P',
       'deposit_type_No Deposit', 'deposit_type_Non Refund',
       'deposit_type_Refundable', 'customer_type_Contract',
       'customer_type_Group', 'customer_type_Transient',
       'customer_type_Transient-Party'],
      dtype='object', length=187)
Index(['lead_time', 'arrival_date_week_number', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled',
       ...
       'assigned_room_type_I', 'assigned_room_type_K', 'assigned_room_type_P',
       'deposit_type_No Deposit', 'deposit_type_Non Refund',
       'deposit_type_Refundable', 'custo

In [139]:
col_name = list(set(adr_x.columns) - set(test_dum.columns))
for i in col_name:
    del adr_x[i]

In [144]:
col_name2 = list(set(test_dum.columns) - set(adr_x.columns))
for i in col_name2:
    del test_dum[i]

In [145]:
print(len(adr_x.columns))
print(len(test_dum.columns))

187
187


In [146]:
xgbr = XGBRegressor(booster="gbtree",
                     max_depth = best_d,
                     learning_rate = best_l,
                     n_estimators = best_n,
                     subsample = best_s)
xgbr.fit(adr_x, adr_y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.85,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [197]:
xgbr.feature_importances_

array([6.6369530e-03, 1.3774150e-02, 5.5761249e-03, 5.8152066e-03,
       1.0956683e-02, 3.1506836e-02, 1.3777667e-03, 7.4532647e-03,
       2.8616660e-03, 1.8199550e-03, 6.7326762e-03, 9.5004132e-03,
       3.5285717e-03, 3.0037332e-03, 2.1850795e-03, 1.1284124e-03,
       9.0974964e-02, 0.0000000e+00, 1.2518783e-02, 5.1688873e-03,
       2.3670707e-02, 9.0216063e-03, 4.8934165e-03, 0.0000000e+00,
       1.0334532e-03, 6.9711015e-05, 2.2506583e-03, 9.5024059e-04,
       4.1521087e-04, 1.0264964e-03, 9.1743201e-04, 5.2519585e-04,
       2.0106696e-03, 0.0000000e+00, 1.8946000e-04, 5.7968678e-04,
       0.0000000e+00, 2.9651821e-04, 4.2553974e-04, 1.1271430e-03,
       2.5038737e-05, 9.3667296e-04, 1.0507972e-03, 2.0231048e-03,
       1.2520504e-03, 3.6923922e-04, 1.0733109e-03, 7.1446830e-04,
       3.6729802e-04, 3.3613603e-04, 4.4554088e-04, 2.1954556e-03,
       3.0681456e-04, 1.2097117e-03, 1.3870059e-03, 1.0077368e-03,
       2.8511899e-04, 8.3202252e-04, 3.7468495e-04, 2.4051864e

In [199]:
feature_name = []
for col,score in zip(adr_x.columns, xgbr.feature_importances_):
    feature_name.append((col,score))
sorted_coef = sorted(feature_name, key=lambda x: abs(x[1]), reverse = True)

In [201]:
print(sorted_coef[:20])

[('deposit_type_No Deposit', 0.14591478), ('market_segment_Online TA', 0.12571748), ('hotel_City Hotel', 0.090974964), ('market_segment_Direct', 0.056989245), ('reserved_room_type_A', 0.05174514), ('market_segment_Complementary', 0.04813082), ('children', 0.031506836), ('reserved_room_type_F', 0.027055608), ('deposit_type_Non Refund', 0.02436777), ('meal_HB', 0.023670707), ('reserved_room_type_B', 0.018198721), ('arrival_date_week_number', 0.01377415), ('distribution_channel_Direct', 0.013498976), ('assigned_room_type_I', 0.013197279), ('market_segment_Groups', 0.013195473), ('market_segment_Offline TA/TO', 0.013151144), ('assigned_room_type_G', 0.012800005), ('meal_BB', 0.012518783), ('reserved_room_type_G', 0.012308519), ('adults', 0.010956683)]


In [203]:
print(sorted_coef)

[('deposit_type_No Deposit', 0.14591478), ('market_segment_Online TA', 0.12571748), ('hotel_City Hotel', 0.090974964), ('market_segment_Direct', 0.056989245), ('reserved_room_type_A', 0.05174514), ('market_segment_Complementary', 0.04813082), ('children', 0.031506836), ('reserved_room_type_F', 0.027055608), ('deposit_type_Non Refund', 0.02436777), ('meal_HB', 0.023670707), ('reserved_room_type_B', 0.018198721), ('arrival_date_week_number', 0.01377415), ('distribution_channel_Direct', 0.013498976), ('assigned_room_type_I', 0.013197279), ('market_segment_Groups', 0.013195473), ('market_segment_Offline TA/TO', 0.013151144), ('assigned_room_type_G', 0.012800005), ('meal_BB', 0.012518783), ('reserved_room_type_G', 0.012308519), ('adults', 0.010956683), ('agent', 0.009500413), ('meal_SC', 0.009021606), ('assigned_room_type_H', 0.008963132), ('is_repeated_guest', 0.0074532647), ('booking_changes', 0.006732676), ('lead_time', 0.006636953), ('reserved_room_type_H', 0.0060193506), ('distribution

In [147]:
ypred_adr = xgbr.predict(test_dum)

In [149]:
print(type(ypred_adr))

<class 'numpy.ndarray'>


In [225]:
op_test = [1]*test.shape[0]
s_test = ypred_adr*(op_test - ypred_can)*(test["stays_in_weekend_nights"] + test["stays_in_week_nights"])
arr_df_test = pd.DataFrame(arr_date_test)
cat_test = pd.concat([arr_df_test, s_test], axis = 1)
cat_test.columns = ["arrDate", "rev"]
print(cat_test)

          arrDate          rev
0      2017-04-01   683.343918
1      2017-04-01     0.000000
2      2017-04-01   477.992889
3      2017-04-01   903.617264
4      2017-04-01     0.000000
...           ...          ...
27854  2017-08-31  1025.745667
27855  2017-08-31  1773.361389
27856  2017-08-31   621.153717
27857  2017-08-31   658.189537
27858  2017-08-31    95.569763

[27859 rows x 2 columns]


In [151]:
test_nl = pd.read_csv("test_nolabel.csv")

In [152]:
uni_date_test = unique(arr_date_test)
print(len(uni_date_test))
print(len(test_nl))

153
153


In [226]:
blank_test = [0]*len(uni_date_test)
#sum up
for i in range(cat_test.shape[0]):
    for j in range(len(uni_date_test)):
        if cat_test["arrDate"][i] == uni_date_test[j]:
            blank_test[j] += cat_test["rev"][i]
            break

In [227]:
dtrain = xgb.DMatrix(xcl.to_numpy().reshape(-1,1), label=ycl)
dtest = xgb.DMatrix(np.array(blank_test).reshape(-1,1))

In [228]:
param = {'max_depth':best_d_cl, 'eta':1, 'objective':'multi:softmax', 'num_class':10, 
         'learning_rate':best_l_cl, 'subsample':best_s_cl, 'booster':"gbtree", 'eval_metric':'mlogloss'}
num_round = best_n_cl
bst = xgb.train(param, dtrain, num_round)

In [229]:
ypred_cl = bst.predict(dtest)

In [230]:
print(ypred_cl)

[3. 2. 3. 1. 3. 2. 1. 4. 4. 3. 3. 2. 4. 3. 3. 4. 3. 1. 3. 2. 3. 3. 3. 3.
 1. 1. 2. 5. 6. 3. 4. 3. 2. 3. 4. 3. 3. 5. 3. 2. 4. 3. 3. 4. 3. 2. 2. 3.
 3. 4. 4. 4. 2. 3. 4. 3. 2. 5. 3. 2. 2. 3. 4. 3. 4. 5. 3. 2. 4. 3. 4. 6.
 4. 2. 3. 4. 3. 4. 4. 4. 3. 3. 3. 3. 3. 4. 4. 3. 3. 2. 3. 5. 3. 4. 3. 2.
 3. 3. 4. 6. 5. 4. 3. 4. 4. 8. 6. 9. 4. 3. 4. 4. 6. 6. 9. 4. 4. 4. 5. 6.
 6. 6. 5. 5. 6. 5. 5. 9. 8. 4. 4. 5. 4. 9. 6. 6. 5. 6. 9. 6. 6. 6. 6. 3.
 5. 5. 4. 9. 5. 6. 3. 2. 4.]


In [196]:
print(ypred_cl)

[4. 3. 5. 3. 4. 5. 3. 6. 8. 5. 3. 3. 6. 5. 6. 6. 5. 2. 4. 3. 6. 6. 5. 4.
 2. 6. 4. 6. 8. 4. 6. 4. 4. 6. 6. 6. 4. 6. 4. 4. 5. 4. 6. 5. 8. 4. 4. 5.
 9. 9. 5. 5. 3. 5. 6. 4. 4. 8. 4. 5. 3. 5. 6. 6. 5. 6. 3. 3. 8. 6. 5. 8.
 6. 3. 4. 6. 5. 8. 5. 9. 5. 5. 5. 5. 6. 5. 6. 6. 5. 4. 5. 9. 5. 8. 5. 4.
 5. 4. 5. 9. 9. 6. 5. 5. 6. 8. 9. 8. 5. 3. 5. 6. 8. 8. 8. 5. 6. 6. 6. 8.
 9. 8. 8. 6. 8. 6. 8. 8. 8. 6. 6. 6. 6. 8. 8. 8. 6. 8. 8. 8. 8. 8. 8. 4.
 6. 6. 6. 8. 9. 8. 5. 4. 5.]


In [167]:
print(ypred_cl)

[4. 3. 5. 3. 4. 5. 3. 6. 8. 5. 3. 3. 6. 5. 6. 6. 5. 2. 4. 3. 6. 6. 5. 4.
 2. 6. 4. 6. 8. 4. 6. 4. 4. 6. 6. 6. 4. 6. 4. 4. 5. 4. 6. 5. 8. 4. 4. 5.
 9. 9. 5. 5. 3. 5. 6. 4. 4. 8. 4. 5. 3. 5. 6. 6. 5. 6. 3. 3. 8. 6. 5. 8.
 6. 3. 4. 6. 5. 8. 5. 9. 5. 5. 5. 5. 6. 5. 6. 6. 5. 4. 5. 9. 5. 8. 5. 4.
 5. 4. 5. 9. 9. 6. 5. 5. 6. 8. 9. 8. 5. 3. 5. 6. 8. 8. 8. 5. 6. 6. 6. 8.
 9. 8. 8. 6. 8. 6. 8. 8. 8. 6. 6. 6. 6. 8. 8. 8. 6. 8. 8. 8. 8. 8. 8. 4.
 6. 6. 6. 8. 9. 8. 5. 4. 5.]


In [231]:
ypred_df = pd.DataFrame(ypred_cl)
date_df = pd.DataFrame(uni_date_test)
final = pd.concat([date_df, ypred_df], axis = 1)
final.columns = ["arrival_date", "label"]

In [232]:
display(final)

,arrival_date,label
0,2017-04-01,3.0
1,2017-04-02,2.0
2,2017-04-03,3.0
3,2017-04-04,1.0
4,2017-04-05,3.0
...,...,...
148,2017-08-27,5.0
149,2017-08-28,6.0
150,2017-08-29,3.0
151,2017-08-30,2.0


In [233]:
final.to_csv("test_label.csv", index = False, header = True)